## Setup

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

Error importing in API mode: ImportError("dlopen(/Users/wongpeiting/.pyenv/versions/3.13.9/lib/python3.13/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <B96A8100-FA7A-3EFC-8726-931D26646DE6> /Users/wongpeiting/.pyenv/versions/3.13.9/lib/python3.13/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.


In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## TidyCensus Example

This notebook grabs data from the US Census using the [tidycensus package in R](https://walker-data.com/tidycensus/index.html). Make sure you have installed the `tidyverse` and `tidycensus` packages.

In [3]:
%%R 

library(tidycensus)
library(tidyverse)

# store any shapefiles we grab from the census in ~/Library/Caches/tigris
# in case the government removes the shapefiles FTP server again 😭
options(tigris_use_cache = TRUE)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.2
✔ purrr     1.2.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


### Step 1 \| Load Census API key from `.env` file

The `.env` file should look like this:

```bash
CENSUS_API_KEY="your-api-key-here"
```

This is what is known as an environment variable. 

We load it below in Python

In [4]:
from dotenv import load_dotenv
load_dotenv() # <- searches for a file named .env and loads the environment variables in it

True

And in this cell we pass the environment variable to tidycensus in R

In [5]:
%%R 

census_api_key(Sys.getenv("CENSUS_API_KEY")) # <- Loads CENSUS_API_KEY from environment variable and sets it in R

To install your API key for use in future sessions, run this function with `install = TRUE`.


### Step 2 \| Figure out what tables you want data from and at what geographic granularity

Use <https://censusreporter.org/> to figure out which tables you want.

-   Scroll to the bottom of the page to see the tables.
-   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

Note, you probably always want population variable (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

- find some other variables that you're also interested in
- don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
).

👉 use the code from the example notebooks to get the variables you're interested in. feel free to use ChatGPT for help

In [ ]:
%%R 

library(sf)
library(tigris)

# TOPIC: Unmarried Births in NYC
#
# Table B13014: Women 15-50 Who Had a Birth by Marital Status and Educational Attainment
# - B13014_002: Women who had a birth in past 12 months (DENOMINATOR)
# - B13014_009: Unmarried women who had a birth (NUMERATOR)
#
# Source: https://censusreporter.org/tables/B13014/

nyc_census_data <- get_acs(
  geography = "tract",
  state = "NY",
  county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
  variables = c(
    population = "B01003_001",        # Total population
    total_births = "B13014_002",      # Women who had a birth (denominator)
    unmarried_births = "B13014_009"   # Unmarried women who had a birth (numerator)
  ),
  year = 2021,
  survey = "acs5",
  geometry = TRUE
)

nyc_census_data

### Step 3 | Pivot to Wide Format

Pivot from long to wide using `pivot_wider`.

Depending on what you're trying to do, you may want the data in long or wide format. This example pivots the data to wide format, which will help us with mapping later on in the notebook.

In [7]:
%%R 

nyc_census_data <- nyc_census_data %>% 
  pivot_wider(
    names_from = variable, 
    values_from = c(estimate, moe),
    names_glue = "{variable}_{.value}"
  )

# Check the structure
nyc_census_data %>%
  st_drop_geometry() %>%
  head()

# A tibble: 6 × 8
  GEOID  NAME  population_estimate poverty_universe_est…¹ poverty_below_estimate
  <chr>  <chr>               <dbl>                  <dbl>                  <dbl>
1 36081… Cens…                2863                   2862                    165
2 36047… Cens…                3655                   3655                    232
3 36061… Cens…                2450                   2126                     89
4 36047… Cens…                2676                   2675                     49
5 36047… Cens…                3004                   3004                    242
6 36047… Cens…                1834                   1834                    141
# ℹ abbreviated name: ¹​poverty_universe_estimate
# ℹ 3 more variables: population_moe <dbl>, poverty_universe_moe <dbl>,
#   poverty_below_moe <dbl>


### Step 4 | Do calculations with `mutate`

In R, you can use `mutate` to create new columns or modify existing ones.

👉 In this step, do any calculations you need (for example if you have the total black population, but you want it as a percent of total population in a census tract, you may have to do some division)

In [ ]:
%%R 

# Calculate percentage of births to unmarried women and extract borough name
nyc_census_data <- nyc_census_data %>% 
  mutate(
    # Percent unmarried = unmarried births / total births * 100
    pct_unmarried = (unmarried_births_estimate / total_births_estimate) * 100,
    
    # Extract borough name from the NAME column
    # NAME looks like "Census Tract 147, Queens County, New York"
    borough = str_split(NAME, ",") %>% map_chr(2) %>% str_trim()
  ) %>%
  # Remove tracts with missing data
  filter(!is.na(pct_unmarried), is.finite(pct_unmarried))

# Summary statistics
cat("% Births to Unmarried Women - Summary:\n")
summary(nyc_census_data$pct_unmarried)

cat("\nNumber of tracts by borough:\n")
nyc_census_data %>%
  st_drop_geometry() %>%
  count(borough)

### Step 5 | Example Plots

Use histograms or dotplots to view the distributions

In [ ]:
%%R -w 1000 -h 400

# Boxplot comparison across boroughs
ggplot(nyc_census_data) +
  aes(x = pct_unmarried, y = reorder(borough, pct_unmarried, FUN = median)) +
  geom_boxplot(fill = "coral", alpha = 0.7, outlier.alpha = 0.3) +
  labs(
    title = "Births to Unmarried Women by Borough",
    subtitle = "Each point is a census tract",
    x = "% of Births to Unmarried Women",
    y = ""
  ) +
  theme_minimal()

In [ ]:
%%R -w 1000 -h 400

# Histogram of unmarried birth rates by borough
ggplot(nyc_census_data) +
  aes(x = pct_unmarried) +
  geom_histogram(bins = 30, fill = "coral", color = "white") +
  facet_wrap(~borough, scales = "free_y") +
  labs(
    title = "Distribution of Unmarried Birth Rates by Borough",
    subtitle = "Census tract level, ACS 2017-2021",
    x = "% of Births to Unmarried Women",
    y = "Number of Tracts"
  ) +
  theme_minimal()

### Step 6 \| Example Map

See more details about making maps here:

https://walker-data.com/tidycensus/articles/spatial-data.html

In [ ]:
%%R -w 1000 -h 1000

# Remove water areas so rivers show on the map
nyc_census_data <- nyc_census_data %>% erase_water()

# Choropleth map of unmarried birth rates
ggplot(nyc_census_data) +
  geom_sf(aes(fill = pct_unmarried), color = NA) +
  scale_fill_viridis_c(
    option = "magma",
    name = "% Births to\nUnmarried Women",
    labels = function(x) paste0(round(x), "%")
  ) +
  theme_void() +
  labs(
    title = "Births to Unmarried Women Across NYC",
    subtitle = "Percentage of women who had a birth in the past 12 months who were unmarried",
    caption = "Source: US Census Bureau, ACS 2017-2021 (Table B13014)"
  )